In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
# Build a SparkSession using the SparkSession APIs.
# If one does not exist, then create an instance. There
# can only be one SparkSession per JVM.
spark = (SparkSession
 .builder
 .appName("PythonMnMCount")
 .getOrCreate())
# Get the M&M data set

In [3]:

    # get the M&M data set file name
    mnmFile = "C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/chapter2/py/src/data/mnm_dataset.csv"
    # read the file into a Spark DataFrame
    mnmDF = (spark.read.format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .load(mnmFile))
    # display DataFrame
    mnmDF.show(5, False)

+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|TX   |Red   |20   |
|NV   |Blue  |66   |
|CO   |Blue  |79   |
|OR   |Blue  |71   |
|WA   |Yellow|93   |
+-----+------+-----+
only showing top 5 rows



In [4]:
# Crear TempView
mnmDF.createOrReplaceTempView("mnmDFView")

In [15]:
    # aggregate count of all colors and groupBy state and color
    # orderBy descending order

    countMnMDF = (mnmDF
     .select("State", "Color", "Count")
     .groupBy("State", "Color")
     .agg(count("Count").alias("Total"))
     .orderBy("Total",ascending=False))

    # show all the resulting aggregation for all the dates and colors
    countMnMDF.show(60)
    print("Total Rows = %d" % (countMnMDF.count()))

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   CA|Yellow| 1807|
|   WA| Green| 1779|
|   OR|Orange| 1743|
|   TX| Green| 1737|
|   TX|   Red| 1725|
|   CA| Green| 1723|
|   CO|Yellow| 1721|
|   CA| Brown| 1718|
|   CO| Green| 1713|
|   NV|Orange| 1712|
|   TX|Yellow| 1703|
|   NV| Green| 1698|
|   AZ| Brown| 1698|
|   CO|  Blue| 1695|
|   WY| Green| 1695|
|   NM|   Red| 1690|
|   AZ|Orange| 1689|
|   NM|Yellow| 1688|
|   NM| Brown| 1687|
|   UT|Orange| 1684|
|   NM| Green| 1682|
|   UT|   Red| 1680|
|   AZ| Green| 1676|
|   NV|Yellow| 1675|
|   NV|  Blue| 1673|
|   WA|   Red| 1671|
|   WY|   Red| 1670|
|   WA| Brown| 1669|
|   NM|Orange| 1665|
|   WY|  Blue| 1664|
|   WA|Yellow| 1663|
|   WA|Orange| 1658|
|   NV| Brown| 1657|
|   CA|Orange| 1657|
|   CO| Brown| 1656|
|   CA|   Red| 1656|
|   UT|  Blue| 1655|
|   AZ|Yellow| 1654|
|   TX|Orange| 1652|
|   AZ|   Red| 1648|
|   OR|  Blue| 1646|
|   OR|   Red| 1645|
|   UT|Yellow| 1645|
|   CO|Orange| 1642|
|   TX| Brown

In [10]:
# Haciendo lo mismo usando tempViews y Spark SQL
spark.sql("""SELECT State, Color, count(*) as Total
             FROM mnmDFView
             GROUP BY State, Color
             ORDER BY Total desc""").show()

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   CA|Yellow| 1807|
|   WA| Green| 1779|
|   OR|Orange| 1743|
|   TX| Green| 1737|
|   TX|   Red| 1725|
|   CA| Green| 1723|
|   CO|Yellow| 1721|
|   CA| Brown| 1718|
|   CO| Green| 1713|
|   NV|Orange| 1712|
|   TX|Yellow| 1703|
|   AZ| Brown| 1698|
|   NV| Green| 1698|
|   WY| Green| 1695|
|   CO|  Blue| 1695|
|   NM|   Red| 1690|
|   AZ|Orange| 1689|
|   NM|Yellow| 1688|
|   NM| Brown| 1687|
|   UT|Orange| 1684|
+-----+------+-----+
only showing top 20 rows



In [23]:
sumMnMDF = (mnmDF.select("State", "Color", "Count")
        .groupBy("State", "Color")
        .sum("Count")
        .orderBy("sum(Count)",ascending=False))

sumMnMDF.show(10,False)

+-----+------+----------+
|State|Color |sum(Count)|
+-----+------+----------+
|CA   |Yellow|100956    |
|WA   |Green |96486     |
|CA   |Brown |95762     |
|TX   |Green |95753     |
|TX   |Red   |95404     |
|CO   |Yellow|95038     |
|NM   |Red   |94699     |
|OR   |Orange|94514     |
|WY   |Green |94339     |
|NV   |Orange|93929     |
+-----+------+----------+
only showing top 10 rows



In [24]:
# Haciendo lo mismo usando tempViews y Spark SQL
spark.sql("""SELECT State, Color, sum(Count) as Suma
             FROM mnmDFView
             GROUP BY State, Color
             ORDER BY sum(Count) desc""").show(10,False)

+-----+------+------+
|State|Color |Suma  |
+-----+------+------+
|CA   |Yellow|100956|
|WA   |Green |96486 |
|CA   |Brown |95762 |
|TX   |Green |95753 |
|TX   |Red   |95404 |
|CO   |Yellow|95038 |
|NM   |Red   |94699 |
|OR   |Orange|94514 |
|WY   |Green |94339 |
|NV   |Orange|93929 |
+-----+------+------+
only showing top 10 rows



In [25]:
   # find the aggregate count for California by filtering

caCountMnNDF = (mnmDF
     .select("State", "Color", "Count")
     .where(mnmDF.State == "CA")
     .groupBy("State", "Color")
     .agg(count("Count").alias("Total"))
     .orderBy("Total",ascending=False))

    # show the resulting aggregation for California
caCountMnNDF.show(10)

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   CA|Yellow| 1807|
|   CA| Green| 1723|
|   CA| Brown| 1718|
|   CA|Orange| 1657|
|   CA|   Red| 1656|
|   CA|  Blue| 1603|
+-----+------+-----+



In [27]:
# Haciendo lo mismo usando tempViews y Spark SQL
spark.sql("""SELECT State, Color, count(*) as Total
             FROM mnmDFView
             WHERE State== 'CA'
             GROUP BY State, Color
             ORDER BY Total desc""").show(10)

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   CA|Yellow| 1807|
|   CA| Green| 1723|
|   CA| Brown| 1718|
|   CA|Orange| 1657|
|   CA|   Red| 1656|
|   CA|  Blue| 1603|
+-----+------+-----+



In [18]:
caSumMnNDF = (mnmDF.select("*")
      .where(mnmDF.State == "CA")
      .groupBy("State", "Color")
      .sum("Count")
      .orderBy("sum(Count)",ascending=False))

caSumMnNDF.show(10,False)

+-----+------+----------+
|State|Color |sum(Count)|
+-----+------+----------+
|CA   |Yellow|100956    |
|CA   |Brown |95762     |
|CA   |Green |93505     |
|CA   |Red   |91527     |
|CA   |Orange|90311     |
|CA   |Blue  |89123     |
+-----+------+----------+



In [31]:
# Haciendo lo mismo usando tempViews y Spark SQL
spark.sql("""SELECT State, Color, sum(Count) as Suma
             FROM mnmDFView
             WHERE State == 'CA'
             GROUP BY State, Color
             ORDER BY Suma desc""").show(10, False)

+-----+------+------+
|State|Color |Suma  |
+-----+------+------+
|CA   |Yellow|100956|
|CA   |Brown |95762 |
|CA   |Green |93505 |
|CA   |Red   |91527 |
|CA   |Orange|90311 |
|CA   |Blue  |89123 |
+-----+------+------+



In [36]:
from pyspark.sql.functions import *
caCountMnNDF = mnmDF.select(max("Count").alias("Max"),min("Count").alias("Min"),avg("Count").alias("Avg"),count("*").alias("Cuenta"))

caCountMnNDF.show(10)

+---+---+-----------------+------+
|Max|Min|              Avg|Cuenta|
+---+---+-----------------+------+
|100| 10|55.00090000900009| 99999|
+---+---+-----------------+------+



In [33]:
# Haciendo lo mismo usando tempViews y Spark SQL
spark.sql("""SELECT max(Count) as Max, min(Count) as Min, avg(Count) as Avg, count(*) as Cuenta
             FROM mnmDFView""").show(10)

+---+---+-----------------+------+
|Max|Min|              Avg|Cuenta|
+---+---+-----------------+------+
|100| 10|55.00090000900009| 99999|
+---+---+-----------------+------+



In [5]:
mnm_df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)



In [21]:
    quijote="C:/Users/alvaro.romero/Big_Data/Ejercicios_Spark/el_quijote.txt"
    qj_df = (spark.read.format("text")
        .option("header", "true")
        .option("inferSchema", "true")
        .load(quijote))
    qj_df.show(n=5,truncate=False)

+----------------------------------------------------------------------------------------------+
|value                                                                                         |
+----------------------------------------------------------------------------------------------+
|DON QUIJOTE DE LA MANCHA                                                                      |
|Miguel de Cervantes Saavedra                                                                  |
|                                                                                              |
|PRIMERA PARTE                                                                                 |
|CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha|
+----------------------------------------------------------------------------------------------+
only showing top 5 rows



In [22]:
qj_df.show()

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPÍTULO 1: Que ...|
|En un lugar de la...|
|Tuvo muchas veces...|
|En resolución, e...|
|historia más cie...|
|Decía él, que e...|
|En efecto, remata...|
|Imaginábase el p...|
|linaje y patria, ...|
|Limpias, pues, su...|
|Capítulo 2: Que ...|
|Hechas, pues, est...|
|Estos pensamiento...|
|Con estos iba ens...|
|Autores hay que d...|
|muertos de hambre...|
+--------------------+
only showing top 20 rows



In [23]:
qj_df.show(5)

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPÍTULO 1: Que ...|
+--------------------+
only showing top 5 rows



In [24]:
qj_df.count()

2186

In [25]:
qj_df.first()

# Devuelve la primera línea

Row(value='DON QUIJOTE DE LA MANCHA')

In [26]:
qj_df.head()

# Devuelve la cabecera

Row(value='DON QUIJOTE DE LA MANCHA')

In [27]:
qj_df.take(5)

# Devuelve las 5 primeras líneas

[Row(value='DON QUIJOTE DE LA MANCHA'),
 Row(value='Miguel de Cervantes Saavedra'),
 Row(value=''),
 Row(value='PRIMERA PARTE'),
 Row(value='CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha')]